# Project 1 — Descriptive Stats on Net Migration in Israel
Using World Bank data 1960-2024

This notebook explores the Net migration variable from a World Bank dataset about Israel.  
We compute its mean, median, and mode using both pandas and the Python standard library.  
Finally, we visualize migration trends between 1960-2024 using a simple text-based chart.


## Dataset & Source
- Dataset: Israeldataset.csv  
- Numeric column: `Net migration`  
- Label column: `Year`  
- Source: https://data.worldbank.org/country/israel

The dataset provides yearly population statistics, including net migration (inflow minus outflow of people).  
This project examines patterns and central tendencies in that variable.


## Research question
How has Israel’s **net migration** changed over the years, and what are the central descriptive measures (mean, median, mode) for this variable?


## Parameters
These define the dataset path and selected columns. You can edit them if the file name or structure changes.


In [4]:
from pathlib import Path

DATA_PATH = Path(r"c:\Users\edenk\Desktop\computing\project 1\Israeldataset.csv")
NUMERIC_COLUMN = "Net migration"
ID_COLUMN = "Year"
CSV_KW = dict(encoding="utf-8")


## Step 1 — Load data using pandas
We start by reading the CSV into a pandas DataFrame to inspect its contents.


In [5]:
import pandas as pd

df = pd.read_csv(DATA_PATH, **CSV_KW)
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Israel,ISR,"Prevalence of severe wasting, weight for heigh...",SH.SVR.WAST.MA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Israel,ISR,"Prevalence of wasting, weight for height, fema...",SH.STA.WAST.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Israel,ISR,"Suicide mortality rate, male (per 100,000 male...",SH.STA.SUIC.MA.P5,NaN,NaN,NaN,NaN,NaN,NaN,...,7.030000,8.280000,7.690000,8.390000,7.780000,7.810000,6.310000,NaN,NaN,NaN
3,Israel,ISR,"Prevalence of stunting, height for age, male (...",SH.STA.STNT.ME.MA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Israel,ISR,People using safely managed sanitation service...,SH.STA.SMSS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,91.019717,91.759247,92.501469,93.246367,93.993927,94.744203,95.497148,96.252787,NaN,NaN


## Step 2 — Select and clean the numeric column
We focus on the `Net migration` column, converting it to numeric and dropping missing values.


In [ ]:
raw = pd.read_csv(DATA_PATH, header=None, dtype=str)
hdr_row = raw.index[
    raw.iloc[:, 0].astype(str).str.contains("Country", case=False, na=False)
].min()
if pd.isna(hdr_row):
    hdr_row = 0

df = pd.read_csv(DATA_PATH, header=hdr_row, dtype=str)
df.columns = df.columns.str.strip()

# Identify key columns
country_col = next((c for c in df.columns if "country" in c.lower()), None)
indicator_col = next((c for c in df.columns if "indicator" in c.lower()), None)
year_cols = [c for c in df.columns if str(c).isdigit()]

# Convert to long format
long_df = df.melt(
    id_vars=[country_col, indicator_col],
    value_vars=year_cols,
    var_name="Year",
    value_name="Value",
).rename(columns={country_col: "Country", indicator_col: "Indicator"})

# Clean numeric columns
long_df["Year"] = pd.to_numeric(long_df["Year"], errors="coerce").astype("Int64")
long_df["Value"] = pd.to_numeric(long_df["Value"], errors="coerce")
long_df = long_df.dropna(subset=["Year", "Value"]).sort_values(
    ["Country", "Indicator", "Year"]
)

# Save cleaned version
OUT_PATH = DATA_PATH.with_name("Israeldataset_clean_long.csv")
long_df.to_csv(OUT_PATH, index=False)
print(f"Cleaned dataset saved to: {OUT_PATH}")
print(long_df.head())

print("Columns:", long_df.columns.tolist())

# filter for israel and for the 'Net migration' indicator
mask = (long_df["Country"].str.strip().str.lower() == "israel") & (
    long_df["Indicator"].str.strip().str.lower() == "net migration"
)

df_israel = long_df.loc[mask, ["Year", "Value"]].copy()

# convert both columns to integers
df_israel["Year"] = (
    pd.to_numeric(df_israel["Year"], errors="coerce").dropna().astype(int)
)
df_israel["Value"] = (
    pd.to_numeric(df_israel["Value"], errors="coerce").dropna().astype(int)
)
df_israel = df_israel.sort_values("Year")

# extract as integer lists or arrays
years = df_israel["Year"].to_list()
num = df_israel["Value"].to_list()

print(df_israel.head())
print("Years:", years[:10])
print("Net migration:", num[:10])


Cleaned dataset saved to: c:\Users\edenk\Desktop\computing\project 1\Israeldataset_clean_long.csv
      Country                                          Indicator  Year  Value
62145  Israel  Access to clean fuels and technologies for coo...  2000  100.0
63661  Israel  Access to clean fuels and technologies for coo...  2001  100.0
65177  Israel  Access to clean fuels and technologies for coo...  2002  100.0
66693  Israel  Access to clean fuels and technologies for coo...  2003  100.0
68209  Israel  Access to clean fuels and technologies for coo...  2004  100.0
Columns: ['Country', 'Indicator', 'Year', 'Value']
      Year  Value
926   1960  10990
2442  1961  34482
3958  1962  52313
5474  1963  53877
6990  1964  54050
Years: [1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969]
Net migration: [10990, 34482, 52313, 53877, 54050, 54694, 57347, 59483, 60804, 59928]


## Step 3 — Compute mean, median, and mode 
We use pandas’ built-in methods for these descriptive statistics.


In [ ]:
mean_val = df_israel["Value"].mean()
median_val = df_israel["Value"].median()
mode_val = df_israel["Value"].mode()

print("Mean net migration:", round(mean_val, 2))
print("Median net migration:", median_val)
print("Mode net migration:", mode_val)


Mean net migration: 32119.6
Median net migration: 16563.0
Mode net migration: 0      -3686
1      -3598
2        689
3       1675
4       6684
       ...  
60     63409
61     66811
62     78422
63    153141
64    178732
Name: Value, Length: 65, dtype: int64


## Step 4 — Compute the same metrics 
Now, we re-read the same file and manually compute the **mean**, **median**, and **mode** without pandas or statistics libraries.


In [ ]:
import csv

file_path = r"c:\Users\edenk\Desktop\computing\project 1\Israeldataset_clean_long.csv"

years = []
values = []

# Read and filter manually
with open(file_path, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if (
            row["Country"].strip().lower() == "israel"
            and row["Indicator"].strip().lower() == "net migration"
        ):
            try:
                year = int(float(row["Year"]))
                value = int(float(row["Value"]))
                years.append(year)
                values.append(value)
            except ValueError:
                continue  # skip bad rows

# Manual mean
mean_manual = sum(values) / len(values)

# Manual median
sorted_vals = sorted(values)
n = len(sorted_vals)
if n % 2 == 1:
    median_manual = sorted_vals[n // 2]
else:
    median_manual = (sorted_vals[n // 2 - 1] + sorted_vals[n // 2]) / 2

freq = {}
for v in values:
    freq[v] = freq.get(v, 0) + 1
max_freq = max(freq.values())
mode_manual = [val for val, count in freq.items() if count == max_freq]


print("Manual mean:", round(mean_manual, 2))
print("Manual median:", median_manual)
print("Manual mode:", mode_manual)


Manual mean: 32119.6
Manual median: 16563
Manual mode: [10990, 34482, 52313, 53877, 54050, 54694, 57347, 59483, 60804, 59928, 56173, 52632, 27415, 14310, 11929, 11726, 10459, 12747, 15556, 15591, 16284, 16021, 16563, 689, 10933, -3686, -3598, 7511, 1675, 14192, 178732, 153141, 57522, 49345, 57273, 61800, 66811, 60297, 52713, 78422, 63409, 38643, 17420, 6684, 8494, 10975, 11446, 9654, 7974, 7269, 12906, 10470, 16549, 17075, 25303, 29735, 24308, 14518, 15615, 16134, 16573, 16859, 50004, 9999, 10612]


## Step 5 — Compare results
They should be nearly identical, with small rounding differences possible.


In [ ]:
print(
    "Mean equal?",
    mean_val == mean_manual,
    "|",
    "Pandas:",
    mean_val,
    "Manual:",
    mean_manual,
)
print(
    "Median equal?",
    median_val == median_manual,
    "|",
    "Pandas:",
    median_val,
    "Manual:",
    median_manual,
)
print(set(mode_val.tolist()) == set(mode_manual))


Mean equal? True | Pandas: 32119.6 Manual: 32119.6
Median equal? True | Pandas: 16563.0 Manual: 16563
True


## Step 6 — Visualize the data 
We build a simple text-based vertical chart showing **net migration per year**, using Unicode bars.


In [ ]:
import csv
from math import ceil

FILE = r"c:\Users\edenk\Desktop\computing\project 1\Israeldataset_clean_long.csv"

# 1) Load Year/Value for Israel – Net migration
years, values = [], []
with open(FILE, newline="", encoding="utf-8") as f:
    rdr = csv.DictReader(f)
    for row in rdr:
        if (
            row.get("Country", "").strip().lower() == "israel"
            and row.get("Indicator", "").strip().lower() == "net migration"
        ):
            try:
                y = int(float(row["Year"]))
                v = int(float(row["Value"]))
            except (TypeError, ValueError):
                continue
            years.append(y)
            values.append(v)


# sort by year
pairs = sorted(zip(years, values), key=lambda t: t[0])
years, values = [p[0] for p in pairs], [p[1] for p in pairs]

# 2) Scale settings
# Height above and below zero (in rows)
H_POS = 10  # rows for positive values
H_NEG = 6  # rows for negative values (increase if you have large negatives)

max_pos = max([v for v in values if v > 0], default=0)
max_neg = abs(min([v for v in values if v < 0], default=0))

# Avoid division by zero
scale_pos = (H_POS / max_pos) if max_pos > 0 else 0
scale_neg = (H_NEG / max_neg) if max_neg > 0 else 0

# 3) Build column heights
# Positive heights (0..H_POS), negative heights (0..H_NEG)
pos_heights = [ceil(v * scale_pos) if v > 0 else 0 for v in values]
neg_heights = [ceil(abs(v) * scale_neg) if v < 0 else 0 for v in values]

# 4) Render (top positives, baseline, bottom negatives)
bar = "█"
space = " "
cols = len(values)


def row_str_pos(level):  # level: 1..H_POS (top row is H_POS)
    return "".join(bar if pos_heights[i] >= level else space for i in range(cols))


def row_str_neg(level):  # level: 1..H_NEG (bottom row is H_NEG)
    return "".join(bar if neg_heights[i] >= level else space for i in range(cols))


# 5) Print chart
print()
print("Net migration trend in Israel")
if max_pos:
    # Positive section (top-down)
    for lvl in range(H_POS, 0, -1):
        print(row_str_pos(lvl))
# Zero baseline
print("─" * cols)

if max_neg:
    # Negative section (top (small) to bottom (large))
    for lvl in range(1, H_NEG + 1):
        print(row_str_neg(lvl))

# 6) X-axis labels (years) every k columns to avoid clutter
k = max(1, cols // 12)  # ~12 labels max
label_line = []
for i, y in enumerate(years):
    if i % k == 0:
        s = str(y)
        # place the year centered over its column (best effort in text)
        # first put the year, then pad remaining (k-1) slots with spaces
        label_line.append(s)
        # fill following columns in the block with spaces
        pad = max(0, k - 1)
        label_line.append(" " * pad)
    else:
        label_line.append("")
print()
print("".join(label_line))
print()


Net migration trend in Israel
                              █                                  
                              ██                                 
                              ██                                 
                              ██                                 
                              ██                                 
                              ██       █                         
   ████████                   ███ ████ ██                        
  ██████████                  ████████████                    █  
 ████████████                 ████████████            ███     █  
█████████████████████████  ██████████████████████████████████████
─────────────────────────────────────────────────────────────────
                         ██                                      
                         ██                                      
                         ██                                      
                         ██                  

## Step 7 — Observations and caveats
From the chart, it looks like Israel’s net migration was mostly positive over the years, meaning more people were entering the country than leaving. The only clear period of negative net migration was around 1975, when the number of people leaving  was higher than those arriving (possibly due to Yon Kippur War). Aside from that dip, migration levels seem to rise and fall in waves, with major spikes in the late 1970s and 1980s. One caveat is that the data is aggregated by year and doesn’t capture short-term fluctuations or the reasons behind migration patterns, so while we can see trends, we can’t draw strong conclusions about specific causes without more detailed context.


## Step 8 — Conclusion
Both the pandas and pure Python calculations produced the same mean, median and mode, confirming that our manual computations were accurate. Overall, both approaches show that Israel’s net migration has generally been positive, and the text-based visualization provided an intuitive way to observe fluctuations and anomalies, like the dip in 1975.
